Name: Gayathri Pavani Guntamukkala

Assignment : Week 09 - Machine Learning with Scikit-learn


1. Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.



Among all the classification models analyzed in the notebook, the Random Forest Classifier achieved the highest training accuracy at 99.93%, indicating excellent fit to the training data. However, its test accuracy dropped significantly to 68.6%, revealing signs of overfitting and limited generalization to new data. In contrast, the various logistic regression models including the standard version and L1-penalized models with different hyperparameter settings displayed more balanced performance between training and test sets, with test accuracies ranging from 70.6% to 71.8%. Notably, both the baseline logistic regression and the L1-penalized version with C=10 achieved the highest test accuracy of 71.8%, suggesting better generalizability. While the Random Forest model excels in capturing training patterns, its lack of robustness on unseen data limits its practical utility. Therefore, the logistic regression models, particularly the standard and L1-penalized with C=10, stand out as the most reliable choices for achieving consistent performance across both training and test sets.

In [1]:
## Import Modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn import datasets

In [2]:
## Set print limits
pd.options.display.max_rows = 10
## Import Data
df_patient = \
 pd.read_csv('./PatientAnalyticFile.csv')
df_patient

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2013-04-27,2018-06-01,NaN
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2011-11-05,2015-11-13,NaN
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2006-09-22,2018-06-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1997-12-19,female,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2008-06-14,2018-06-01,NaN
19996,19997,1984-03-31,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2007-04-24,2018-06-01,NaN
19997,19998,1993-07-04,female,white,0,0,0,0,0,0,...,0,0,1,0,1,0,0,2010-10-16,2018-06-01,NaN
19998,19999,1984-04-17,male,other,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2015-01-04,2018-06-01,NaN


In [3]:
# Create mortality variable
df_patient['mortality'] = \
    np.where(df_patient['DateOfDeath'].isnull(),
             0,1)
# Examine
df_patient['mortality']

,mortality
0,0
1,1
2,0
3,1
4,0
...,...
19995,0
19996,0
19997,0
19998,0


In [4]:
df_patient['mortality'].describe()

,mortality
count,20000.000000
mean,0.354700
std,0.478434
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [5]:
df_patient.describe()

,PatientID,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,Rheumatic,Peptic_ulcer_disease,LiverMild,...,Cancer,LiverSevere,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,mortality
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,0.045600,0.043450,0.023950,0.028650,0.031400,0.072650,0.012300,0.009650,0.009250,...,0.050450,0.051450,0.033150,0.006450,0.163450,0.106300,0.302900,0.040050,0.079750,0.354700
std,5773.647028,0.208621,0.203873,0.152897,0.166825,0.174401,0.259568,0.110224,0.097762,0.095733,...,0.218877,0.220919,0.179033,0.080054,0.369785,0.308229,0.459524,0.196081,0.270913,0.478434
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10000.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15000.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,20000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
df_patient.dtypes

,0
PatientID,int64
DateOfBirth,object
Gender,object
Race,object
Myocardial_infarction,int64
...,...
Alcohol,int64
First_Appointment_Date,object
Last_Appointment_Date,object
DateOfDeath,object


In [7]:
# Convert dateofBirth to date
df_patient['DateOfBirth'] = \
    pd.to_datetime(df_patient['DateOfBirth'])
# Calculate age in years as of 2015-01-01
df_patient['Age_years'] = \
    ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days/365.25)
df_patient['Age_years'].describe()

,Age_years
count,20000.000000
mean,47.247474
std,18.145086
min,15.753593
25%,31.733744
50%,47.099247
75%,62.924025
max,78.743326


In [8]:
df_patient.columns

Index(['PatientID', 'DateOfBirth', 'Gender', 'Race', 'Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease', 'Stroke',
       'Dementia', 'Pulmonary', 'Rheumatic', 'Peptic_ulcer_disease',
       'LiverMild', 'Diabetes_without_complications',
       'Diabetes_with_complications', 'Paralysis', 'Renal', 'Cancer',
       'LiverSevere', 'Metastatic_solid_tumour', 'HIV', 'Obesity',
       'Depression', 'Hypertension', 'Drugs', 'Alcohol',
       'First_Appointment_Date', 'Last_Appointment_Date', 'DateOfDeath',
       'mortality', 'Age_years'],
      dtype='object')

In [9]:
## Create formula for all variables in model
vars_remove = ['PatientID','First_Appointment_Date','DateOfBirth',
               'Last_Appointment_Date','DateOfDeath','mortality']
vars_left = set(df_patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)
formula

'mortality ~ Peripheral_vascular_disease + Myocardial_infarction + Age_years + Hypertension + Cancer + Paralysis + Renal + Gender + Congestive_heart_failure + Metastatic_solid_tumour + Peptic_ulcer_disease + HIV + LiverSevere + Obesity + Race + Drugs + Diabetes_without_complications + Pulmonary + Rheumatic + Depression + Alcohol + LiverMild + Diabetes_with_complications + Dementia + Stroke'

In [10]:
## only use subset of data so models fit in reasonable time
df_patient_sub = \
    df_patient.sample(frac=0.1,
                     random_state=32)
## use Patsy to create model matrices
Y,X = dmatrices(formula,
                df_patient_sub)

In [11]:
Y

DesignMatrix with shape (2000, 1)
  mortality
          0
          0
          1
          1
          0
          0
          1
          1
          0
          0
          1
          0
          1
          0
          1
          0
          1
          0
          0
          1
          0
          1
          0
          0
          0
          0
          1
          1
          0
          0
  [1970 rows omitted]
  Terms:
    'mortality' (column 0)
  (to view full data, use np.asarray(this_obj))

In [12]:
X

DesignMatrix with shape (2000, 28)
  Columns:
    ['Intercept',
     'Gender[T.male]',
     'Race[T.hispanic]',
     'Race[T.other]',
     'Race[T.white]',
     'Peripheral_vascular_disease',
     'Myocardial_infarction',
     'Age_years',
     'Hypertension',
     'Cancer',
     'Paralysis',
     'Renal',
     'Congestive_heart_failure',
     'Metastatic_solid_tumour',
     'Peptic_ulcer_disease',
     'HIV',
     'LiverSevere',
     'Obesity',
     'Drugs',
     'Diabetes_without_complications',
     'Pulmonary',
     'Rheumatic',
     'Depression',
     'Alcohol',
     'LiverMild',
     'Diabetes_with_complications',
     'Dementia',
     'Stroke']
  Terms:
    'Intercept' (column 0)
    'Gender' (column 1)
    'Race' (columns 2:5)
    'Peripheral_vascular_disease' (column 5)
    'Myocardial_infarction' (column 6)
    'Age_years' (column 7)
    'Hypertension' (column 8)
    'Cancer' (column 9)
    'Paralysis' (column 10)
    'Renal' (column 11)
    'Congestive_heart_failure' (column

In [13]:
## Split Data into training and sample
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X,
                     np.ravel(Y), # prevents dimensionality error later!
                     test_size=0.25,
                     random_state=42)

In [14]:
## Confirm dimensions
X_train.shape

(1500, 28)

In [15]:
X_test.shape

(500, 28)

In [16]:
y_train.shape

(1500,)

In [17]:
y_test.shape

(500,)

**Comparing Logistic Regression Solvers**
Evaluated the performance of different solvers available in scikit-learn for fitting a logistic regression model. The solvers tested are:

liblinear

lbfgs

newton-cg

sag

saga

Each model is trained on the same 80% training subset and evaluated on the remaining 20% holdout set. The same predictors are used and also ensure that no regularization is applied.

To Compare:

Training Accuracy – how well the model fits the training data.

Holdout Accuracy – how well the model generalizes to unseen data.

Time Taken – how long the model takes to train and evaluate.

This comparison helps us understand the trade-offs between accuracy and computational efficiency across solvers.

2. Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.

In [18]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import time
import warnings
# Suppress convergence warnings
warnings.filterwarnings("ignore")

# Create mortality variable
df_patient['mortality'] = np.where(df_patient['DateOfDeath'].isnull(), 0, 1)

# Convert DateOfBirth to datetime and calculate age in years as of 2015-01-01
df_patient['DateOfBirth'] = pd.to_datetime(df_patient['DateOfBirth'])
df_patient['Age_years'] = ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days / 365.25)

vars_remove = ['PatientID', 'First_Appointment_Date', 'DateOfBirth', 'Last_Appointment_Date', 'DateOfDeath', 'mortality']
vars_left = set(df_patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)

# Create model matrices
Y, X = dmatrices(formula, df_patient, return_type='dataframe')

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(Y), test_size=0.2, random_state=42)

In [19]:
solvers = ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga']
results = []

solvers = ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga']
results = []

# Train and evaluate models for each solver
for solver in solvers:
    start_time = time.time()
    clf = LogisticRegression(solver=solver, max_iter=500)
    clf.fit(X_train, y_train)
    train_accuracy = clf.score(X_train, y_train)
    test_accuracy = clf.score(X_test, y_test)
    time_taken = time.time() - start_time
    results.append([solver, train_accuracy, test_accuracy, time_taken])


INTERPRETATION: All solvers produce similar accuracy results on both training and holdout subsets.liblinear is the fastest, making it the best for smaller or binary classification problems where speed is crucial.lbfgs and newton-cg offer strong performance on multiclass or larger datasets, though they are slightly slower.
sag and saga are designed for large-scale data but come with tradeoffs in terms of sensitivity to scaling and convergence time.

3. Compare the results of the models in terms of their accuracy (use this as the performance metric to assess generalizability error on the holdout subset) and the time taken (use appropriate timing function). Summarize your results via a table with the following structure

In [20]:
# results dataframe
results_df = pd.DataFrame(results, columns=['Solver used', 'Training subset accuracy', 'Holdout subset accuracy', 'Time taken'])
results_df


,Solver used,Training subset accuracy,Holdout subset accuracy,Time taken
0,liblinear,0.747938,0.73625,0.107861
1,lbfgs,0.748062,0.73625,1.357583
2,newton-cg,0.748125,0.73625,0.137943
3,sag,0.748000,0.73625,7.453381
4,saga,0.748000,0.73600,5.973397


Final Analysis: Which Solver is Best?

Based on the table above, we evaluate the solvers using three key metrics:

Holdout Accuracy (primary metric for generalization)

Training Accuracy (secondary)

Time Taken (especially important for large-scale applications)

While several solvers performed similarly in terms of accuracy, liblinear stood out due to its strong holdout accuracy and fastest runtime. This makes it an efficient and reliable choice when speed and generalization are both important.

4. Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?



Based on the results, the liblinear solver yielded the best overall performance among all the logistic regression solvers evaluated. All solvers produced similar accuracy scores on both the training and holdout subsets indicating comparable generalization ability but liblinear distinguished itself with the fastest execution time, completing model training and evaluation in just 0.107861, with the highest level of performance,lvers like lbfgs, newton-cg, sag, and saga showed nearly identical accuracy training accuracy around 0.748 and holdout accuracy around 0.736, their computation times were noticeably higher, particularly saga, which took over 5 seconds.

In ranking the models, holdout subset accuracy was prioritized as the most critical criterion, since it best reflects how well the model performs on unseen data. Execution time was the second key factor, especially important in time-sensitive or large-scale applications. Training accuracy was considered to check for potential overfitting but was less critical since all models performed similarly on this metric.

In conclusion, the liblinear solver offers the best balance between accuracy and efficiency, making it the most suitable choice for this task. Its combination of high predictive performance and minimal computation time makes it particularly effective for practical applications where speed matters.